## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
# from config import g_key
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()

g_key = os.getenv("goog_key")
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [22]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Yellowknife,CA,62.4560,-114.3525,61.32,67,20,3.44,few clouds
1,1,Khatanga,RU,71.9667,102.5000,60.03,86,100,11.74,moderate rain
2,2,Ushuaia,AR,-54.8000,-68.3000,35.26,69,0,4.61,clear sky
3,3,Yumen,CN,40.2833,97.2000,90.70,8,4,15.39,clear sky
4,4,Port Alfred,ZA,-33.5906,26.8910,58.17,85,56,5.01,broken clouds


In [24]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 65


In [25]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Yellowknife,CA,62.4560,-114.3525,61.32,67,20,3.44,few clouds
1,1,Khatanga,RU,71.9667,102.5000,60.03,86,100,11.74,moderate rain
4,4,Port Alfred,ZA,-33.5906,26.8910,58.17,85,56,5.01,broken clouds
6,6,Kaeo,NZ,-35.1000,173.7833,60.33,79,19,9.82,few clouds
8,8,Cape Town,ZA,-33.9258,18.4232,59.38,82,4,8.99,clear sky
9,9,Busselton,AU,-33.6500,115.3333,60.85,64,97,4.03,overcast clouds
10,10,Jamestown,US,42.0970,-79.2353,62.19,66,85,1.01,overcast clouds
13,13,Hermanus,ZA,-34.4187,19.2345,48.79,85,31,6.26,scattered clouds
17,17,New Norfolk,AU,-42.7826,147.0587,56.70,77,67,4.00,light rain
19,19,Bredasdorp,ZA,-34.5322,20.0403,48.36,92,13,4.74,few clouds


In [26]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().count()

City_ID        198
City           198
Country        198
Lat            198
Lng            198
Max Temp       198
Humidity       198
Cloudiness     198
Wind Speed     198
Description    198
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Yellowknife,CA,61.32,few clouds,62.4560,-114.3525,
1,Khatanga,RU,60.03,moderate rain,71.9667,102.5000,
4,Port Alfred,ZA,58.17,broken clouds,-33.5906,26.8910,
6,Kaeo,NZ,60.33,few clouds,-35.1000,173.7833,
8,Cape Town,ZA,59.38,clear sky,-33.9258,18.4232,
9,Busselton,AU,60.85,overcast clouds,-33.6500,115.3333,
10,Jamestown,US,62.19,overcast clouds,42.0970,-79.2353,
13,Hermanus,ZA,48.79,scattered clouds,-34.4187,19.2345,
17,New Norfolk,AU,56.70,light rain,-42.7826,147.0587,
19,Bredasdorp,ZA,48.36,few clouds,-34.5322,20.0403,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    try:  
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Yellowknife,CA,61.32,few clouds,62.4560,-114.3525,The Explorer Hotel
1,Khatanga,RU,60.03,moderate rain,71.9667,102.5000,"""Khatanga"""
4,Port Alfred,ZA,58.17,broken clouds,-33.5906,26.8910,The Halyards Hotel
6,Kaeo,NZ,60.33,few clouds,-35.1000,173.7833,Bushido Lodge
8,Cape Town,ZA,59.38,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
...,...,...,...,...,...,...,...
657,Aurich,DE,57.94,scattered clouds,53.4667,7.4833,Ringhotel Köhlers Forsthaus
659,College,US,59.32,broken clouds,64.8569,-147.8028,Pike's Waterfront Lodge
665,Chilca,PE,59.76,overcast clouds,-12.5211,-76.7372,Playa Punta Yaya - Las Salinas
670,Buenos Aires,AR,64.98,light rain,-34.6132,-58.3772,Hotel NH Buenos Aires City


In [31]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [5]:
# clean_hotel_df = pd.read_csv("WeatherPy_vacation.csv")
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [6]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))